# Übung 2 vom 18.10.2022

### Exercise 1

Entwerfen Sie einen Algorithmus zur gerechten Aufteilung eines Kuchens unter 3 Personen. Beschreiben Sie den Algorithmus in geeigneter Weise, zum Beispiel in Pseudocode
oder als Nassi-Shneidermann Diagramm. Auch bebilderter Fließtext kann sinnvoll sein.

In [827]:
from dataclasses import dataclass, field
import random as random

@dataclass
class Person:
    name: str
    estimate_percent: float = 0
    percent: float = 0
    cake: float = 0
    position: int = field(init=False, default=0, repr=False)

    def __str__(self) -> str:
        return f"{self.position}. Aufteilung: {self.name} schätzt {round(self.estimate_percent*100,2)}% und bekommt {round(self.cake,2)} ({round((self.cake/Group.cake*100),2)}%) Kuchen ({round(self.percent*100,2)}%, diff {round((self.percent-self.estimate_percent)*100,2)})"

@dataclass
class Group:
    name: str
    members: list[Person]
    cake: float = 200

    def __init__(self, name: str, members: list[Person]) -> None:
        self.name = name
        self.members = members

    def __str__(self) -> str:
        # sort members by estimate_percent
        members_sorted = sorted(self.members, key=lambda member: member.position)
        mermbers_str = "\n  - ".join([str(member) for member in members_sorted])
        member_highest_percent = max(self.members, key=lambda member: member.percent)
        return f"Gruppe: {self.name}\n- Mitglieder: {len(self.members)}\n  - {mermbers_str}\n\nGrößtes Stück: {member_highest_percent}"


In [828]:
members = [
    Person("Alice"),
    Person("Bob"),
    Person("Charlie"),
    Person("David"),
    # Person("Eve"),
    # Person("Frank"),
    # Person("Grace"),
    # Person("Hans"),
]

### Gerecht, aber nicht neidfrei (wahrscheinlich falsch)

Die Variable `estimate_variance` definiert wie gut die Personen bei ihrer Schätzung sind.

In [829]:
group1 = Group("Gerechte Aufteilung", members.copy())

estimate_variance = 0.05

while group1.cake > 0:
    
    # filter members who already have a slice
    members_filtered = [member for member in group1.members if member.position == 0]
    
    if len(members_filtered) == 2:
        m1 = members_filtered[len(members_filtered)-1]
        m2 = members_filtered[len(members_filtered)-2]
        # get estimate for last two members
        m1.estimate_percent = 1 / len(members_filtered) + random.uniform(-estimate_variance, estimate_variance)
        m2.estimate_percent = 1 / len(members_filtered) + random.uniform(-estimate_variance, estimate_variance)
        # find mean of estimates
        mean_estimate = (m1.estimate_percent + m2.estimate_percent) / 2
        # split remaining cake equally
        small_part = mean_estimate if mean_estimate < 0.5 else 1 - mean_estimate
        # check which member gets the small part
        # The idea is that the member with the "better" estimate gets the bigger part
        if  mean_estimate < 0.5:
            m1.percent = small_part if m1.estimate_percent < m2.estimate_percent else 1 - small_part
            m2.percent = 1 - small_part if m1.estimate_percent < m2.estimate_percent else small_part
        else :
            m1.percent = small_part if m1.estimate_percent > m2.estimate_percent else 1 - small_part
            m2.percent = 1 - small_part if m1.estimate_percent > m2.estimate_percent else small_part
        # give remaining cake to members
        m1.cake = group1.cake * m1.percent
        m2.cake = group1.cake * m2.percent
        # set position of members
        m1.position = len(group1.members) - len(members_filtered) + 1
        m2.position = len(group1.members) - len(members_filtered) + 2
        # set cake to 0
        group1.cake = 0
        
    else: 
        # get estimate for all remaining members
        for member in members_filtered:
            member.estimate_percent = 1 / len(members_filtered) + random.uniform(-estimate_variance, estimate_variance)

        # find member with smallest estimate
        member_smallest_estimate = min(members_filtered, key=lambda member: member.estimate_percent)
        # find member with second smallest estimate
        member_second_smallest_estimate = min(members_filtered, key=lambda member: member.estimate_percent if member != member_smallest_estimate else 1)
        # find mean of estimates
        mean_estimate = (member_smallest_estimate.estimate_percent + member_second_smallest_estimate.estimate_percent) / 2
        # split remaining cake one time and give it to member with smallest estimate
        member_smallest_estimate.percent = mean_estimate
        member_smallest_estimate.cake = mean_estimate * group1.cake 
        # set position of member
        member_smallest_estimate.position = len(group1.members) - len(members_filtered) + 1
        # reduce cake
        group1.cake -= member_smallest_estimate.cake
       
# print results
print(group1)

Gruppe: Gerechte Aufteilung
- Mitglieder: 4
  - 1. Aufteilung: Alice schätzt 22.39% und bekommt 22.44 (22.44%) Kuchen (22.44%, diff 0.05)
  - 2. Aufteilung: David schätzt 29.09% und bekommt 22.74 (22.74%) Kuchen (29.33%, diff 0.24)
  - 3. Aufteilung: Charlie schätzt 49.83% und bekommt 28.27 (28.27%) Kuchen (51.57%, diff 1.74)
  - 4. Aufteilung: Bob schätzt 47.02% und bekommt 26.54 (26.54%) Kuchen (48.43%, diff 1.4)

Größtes Stück: 3. Aufteilung: Charlie schätzt 49.83% und bekommt 28.27 (28.27%) Kuchen (51.57%, diff 1.74)


### Neidfrei